In [ ]:
!pip install -q google-generativeai==0.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 15.0 MB/s eta 0:00:00


In [ ]:
!pip install -q tenacity

In [ ]:
!pip install -q ranx

In [ ]:
import google.generativeai as genai
from pathlib import Path
import sqlite3
import tenacity

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
# prompt: from path /content/drive/MyDrive/Admission, lấy ra các file xlsx và in ra tên nó đi

import os

path = "/content/drive/MyDrive/University_Excel_File"

for filename in os.listdir(path):
    if filename.endswith(".xlsx"):
      print(filename)

PNTU.xlsx
OU.xlsx
NTTU.xlsx
HCMUTE.xlsx
HCMUS.xlsx
HCMUE.xlsx
FTU2.xlsx
VLU.xlsx
UMP.xlsx
UIT.xlsx
UFM.xlsx
UEH.xlsx
FPT.xlsx
TDTU.xlsx
BKU.xlsx


# read each files and solve to build ground truth.

In [ ]:
# prompt: read and view col in file. /content/drive/MyDrive/Admission/TDTU.xlsx df

import pandas as pd

# Assuming 'df' is already defined from a previous cell or you will define it here
# Example if you haven't defined df yet:
excel_file_path = '/content/drive/MyDrive/Admission/TDTU.xlsx'
try:
  df = pd.read_excel(excel_file_path)
except FileNotFoundError:
  print(f"Error: File not found at {excel_file_path}")
except Exception as e:
  print(f"An error occurred: {e}")
else:
  # Now you have the dataframe 'df'
  print(df.columns)  # Print all column names

Index(['id', 'text', 'University', 'filename'], dtype='object')


In [ ]:
df.head()

,id,text,University,filename
0,014a418b-fded-4bd1-a1b1-931188a656ef,Thông báo phương án tuyển sinh 2021\n\nTrường ...,TDTU,/content/drive/MyDrive/Data/Admission/TDTU/202...
1,0183c28f-552b-426d-b052-c8c0791f8683,PHƯƠNG THỨC 1: XÉT TUYỂN THEO KẾT QUẢ QUÁ TRÌN...,TDTU,/content/drive/MyDrive/Data/Admission/TDTU/202...
2,027aec04-add2-4a22-80ef-09c0180f1a63,Thời gian nộp hồ sơ xét tuyển: từ 15/4 - 29/5/...,TDTU,/content/drive/MyDrive/Data/Admission/TDTU/202...
3,031818d4-9c44-44d2-beea-580f4e64033e,"Đối với chương trình đại học bằng tiếng Anh, x...",TDTU,/content/drive/MyDrive/Data/Admission/TDTU/202...
4,04a30b2c-6a39-4e29-9391-a683bc6c676b,Xét tuyển theo kết quả học tập 06 học kỳ THPT ...,TDTU,/content/drive/MyDrive/Data/Admission/TDTU/202...


In [ ]:
df['filename'].unique()

array(['/content/drive/MyDrive/Data/Admission/TDTU/2021_tuyển sinh.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2021_điểm chuẩn.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2023_Điểm chuẩn.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2024_Điểm chuẩn.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2023_tuyển sinh.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2024_tuyển sinh.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2022_điểm chuẩn.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2022_tuyển sinh.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/Giới thiệu.md'],
      dtype=object)

In [ ]:
# prompt: lấy ra df1 với df['filename'] == /content/drive/MyDrive/Data/Admission/TDTU/2021_tuyển sinh.md

df1 = df[df['filename'] == '/content/drive/MyDrive/Data/Admission/TDTU/2021_tuyển sinh.md']
df1[:5]

,id,text,University,filename
0,014a418b-fded-4bd1-a1b1-931188a656ef,Thông báo phương án tuyển sinh 2021\n\nTrường ...,TDTU,/content/drive/MyDrive/Data/Admission/TDTU/202...
1,0183c28f-552b-426d-b052-c8c0791f8683,PHƯƠNG THỨC 1: XÉT TUYỂN THEO KẾT QUẢ QUÁ TRÌN...,TDTU,/content/drive/MyDrive/Data/Admission/TDTU/202...
2,027aec04-add2-4a22-80ef-09c0180f1a63,Thời gian nộp hồ sơ xét tuyển: từ 15/4 - 29/5/...,TDTU,/content/drive/MyDrive/Data/Admission/TDTU/202...
3,031818d4-9c44-44d2-beea-580f4e64033e,"Đối với chương trình đại học bằng tiếng Anh, x...",TDTU,/content/drive/MyDrive/Data/Admission/TDTU/202...
4,04a30b2c-6a39-4e29-9391-a683bc6c676b,Xét tuyển theo kết quả học tập 06 học kỳ THPT ...,TDTU,/content/drive/MyDrive/Data/Admission/TDTU/202...


# Load llm

In [ ]:
from google.colab import userdata

genai.configure(api_key = userdata.get('geminai_key_3'))

In [ ]:
# Set up the model
generation_config = {
  "temperature": 0.4,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

In [ ]:
model = genai.GenerativeModel(model_name = "gemini-1.5-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

In [ ]:
# prompt: lấy df1[0]['text'] là input để cho model tạo ra question từ nội dung của nó đi

def generate_question(text):
    prompt = f"""
    Bạn là một trợ lý AI thông minh. Nhiệm vụ của bạn là tạo ra một câu hỏi liên quan trực tiếp đến nội dung của đoạn văn sau. Hãy đảm bảo câu hỏi tập trung vào các ý chính hoặc thông tin quan trọng trong đoạn văn.
    Câu hỏi ngắn gọn thôi, xúc tích vào. Tầm 20-35 token được rồi. Chú ý câu hỏi thường phải hỏi năm trong câu cụ thể có 4 năm là 2021, 2022, 2023, 2024.
    Nội dung:
    "{text}"

    Câu hỏi:
    """
    response = model.generate_content(prompt, stream=False)  # Đảm bảo sử dụng đúng phương thức
    return response.text.strip()  # Loại bỏ khoảng trắng thừa ở kết quả


## Test generate quesiton

In [ ]:
# Example usage
# Access the first row's 'text' value using .iloc
first_text = """Kỹ thuật Nhiệt

Tổng quan

Kỹ thuật Nhiệt tập trung vào việc nghiên cứu và ứng dụng các nguyên lý nhiệt, nhiệt động học và cơ học chất lỏng để thiết kế và phát triển các hệ thống liên quan đến năng lượng và nhiệt, từ các hệ thống lạnh, điều hòa không khí đến các công nghệ năng lượng tái tạo.

CÁC ĐIỂM ĐẶC BIỆT

Ứng dụng năng lượng tái tạo: Sinh viên sẽ học cách thiết kế các hệ thống năng lượng sạch, như năng lượng mặt trời và gió.

Tiết kiệm năng lượng: Ngành này cũng chú trọng đến việc tối ưu hóa việc sử dụng năng lượng, bảo vệ môi trường.

Triển vọng việc làm

Các công ty trong lĩnh vực năng lượng, điều hòa không khí và làm lạnh.

Các dự án nghiên cứu về năng lượng tái tạo và hiệu suất năng lượng.

Các công ty phát triển công nghệ năng lượng bền vững.

Logistics & Quản lý chuỗi cung ứng

Tổng quan"""
print(first_text)
generated_question = generate_question(first_text)
generated_question

Kỹ thuật Nhiệt

Tổng quan

Kỹ thuật Nhiệt tập trung vào việc nghiên cứu và ứng dụng các nguyên lý nhiệt, nhiệt động học và cơ học chất lỏng để thiết kế và phát triển các hệ thống liên quan đến năng lượng và nhiệt, từ các hệ thống lạnh, điều hòa không khí đến các công nghệ năng lượng tái tạo.

CÁC ĐIỂM ĐẶC BIỆT

Ứng dụng năng lượng tái tạo: Sinh viên sẽ học cách thiết kế các hệ thống năng lượng sạch, như năng lượng mặt trời và gió.

Tiết kiệm năng lượng: Ngành này cũng chú trọng đến việc tối ưu hóa việc sử dụng năng lượng, bảo vệ môi trường.

Triển vọng việc làm

Các công ty trong lĩnh vực năng lượng, điều hòa không khí và làm lạnh.

Các dự án nghiên cứu về năng lượng tái tạo và hiệu suất năng lượng.

Các công ty phát triển công nghệ năng lượng bền vững.

Logistics & Quản lý chuỗi cung ứng

Tổng quan


'Ngành Kỹ thuật Nhiệt tập trung vào những ứng dụng năng lượng nào?'

In [ ]:
# Example usage
# Access the first row's 'text' value using .iloc
first_text = df1.iloc[1]['text']
print(first_text)
generated_question = generate_question(first_text)
generated_question

## For 1 df == in filename

In [ ]:
# prompt: setting để output của pd có thể xem toàn bộ

import pandas as pd

# ... (Your existing code) ...

pd.set_option("display.max_rows", None)  # Display all rows
pd.set_option("display.max_columns", None) # Display all columns
pd.set_option("display.width", None) # Auto adjust display width
pd.set_option("display.max_colwidth", None) # Display full column width

# ... (rest of your code) ...

In [ ]:
# prompt: for i in len(df1) và lấy ra từng row và tạo thành 1 pd với id là row và question đã sinh ra

import pandas as pd

# Assuming df1 is already defined from the previous code

data = []
for i in range(len(df1)):
    row = df1.iloc[i]
    text = row['text']
    question = generate_question(text)
    data.append({'id_question': row['University']+ f"{i}", 'question': question, 'id_corpus': row['id']})

new_df = pd.DataFrame(data)
new_df

In [ ]:
import uuid
import time  # Import thư viện time để thêm khoảng nghỉ

data = []
for i in range(len(df1)):
    row = df1.iloc[i]
    text = row['text']
    question = generate_question(text)

    # Tạo UUID cho mỗi id_question
    id_question = str(uuid.uuid4())

    data.append({'id_question': id_question, 'question': question, 'id_corpus': row['id']})

new_df1 = pd.DataFrame(data)
new_df1

Đang nghỉ sau 5 lần yêu cầu...
Đang nghỉ sau 10 lần yêu cầu...
Đang nghỉ sau 15 lần yêu cầu...
Đang nghỉ sau 20 lần yêu cầu...
Đang nghỉ sau 25 lần yêu cầu...
Đang nghỉ sau 30 lần yêu cầu...
Đang nghỉ sau 35 lần yêu cầu...


,id_question,question,id_corpus
0,37ac8dfc-773d-4d5f-9898-6adc29ee51ef,Trường Đại học Tôn Đức Thắng dự kiến tuyển sinh bao nhiêu chỉ tiêu đại học năm 2021?,014a418b-fded-4bd1-a1b1-931188a656ef
1,97ffb5d4-9ae5-4694-ba31-ed4491d8783e,Điểm xét tuyển đợt 1 năm 2021 của TDTU dựa trên kết quả học tập của mấy học kỳ THPT?,0183c28f-552b-426d-b052-c8c0791f8683
2,02c3ef21-0855-45e7-b7db-b70a62077911,Hạn nộp hồ sơ xét tuyển đợt 1 năm 2021 là khi nào?,027aec04-add2-4a22-80ef-09c0180f1a63
3,d3d025a7-c13d-4fcf-b479-813f03cfedb3,Thời gian đăng ký xét tuyển trực tuyến chương trình đại học bằng tiếng Anh năm 2021 là khi nào?,031818d4-9c44-44d2-beea-580f4e64033e
4,ad19a017-610e-4442-99fc-ef171ff24285,Thời gian đăng ký xét tuyển trực tuyến năm 2021 là khi nào?,04a30b2c-6a39-4e29-9391-a683bc6c676b
5,f1b51478-e3ce-475d-8d9d-11581c06ee2a,Điều kiện xét tuyển đặc cách tốt nghiệp THPT năm 2021 tại TDTU là gì?,051a6912-8c83-4f9c-ae8a-bb768448fabc
6,203043c6-823b-45e8-b921-627bd53765c9,"TDTU ưu tiên xét tuyển thí sinh trường chuyên, trọng điểm tốt nghiệp THPT năm nào?",05276d60-e139-45f4-ba11-8fe2e51d0ad4
7,886af518-9aba-414a-bb6c-9188d128364e,Thí sinh đối tượng 1 muốn xét tuyển ngành Kiến trúc năm 2021 cần thi bổ sung môn gì?,06b2ca15-3196-4b3b-bd3e-c2def1055420
8,dc4954b7-85d6-4f34-bc53-ace1342a6f13,Chứng chỉ tiếng Anh quốc tế cần có hiệu lực đến ngày nào để xét tuyển đại học bằng tiếng Anh đợt 1 năm 2021 tại TDTU?,070348b2-66d2-45d7-b766-fbab70038e0d
9,cae49b69-949f-4159-ac7c-10ac21455d0f,Điều kiện đăng ký xét tuyển ưu tiên đợt 2 năm 2021 là gì?,07a9d6c3-4135-4ae8-ae79-cd184de89dd8


In [ ]:


output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output.xlsx"

try:
    new_df1.to_excel(output_path, index=False)  # Use index=False to avoid writing row indices
    print(f"DataFrame successfully exported to {output_path}")
except Exception as e:
    print(f"An error occurred while exporting the DataFrame: {e}")

DataFrame successfully exported to /content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output.xlsx


### 2

In [ ]:
# Lấy danh sách các giá trị unique trong cột 'filename'
docs = df['filename'].unique()

In [ ]:
docs

array(['/content/drive/MyDrive/Data/Admission/TDTU/2021_tuyển sinh.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2021_điểm chuẩn.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2023_Điểm chuẩn.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2024_Điểm chuẩn.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2023_tuyển sinh.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2024_tuyển sinh.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2022_điểm chuẩn.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/2022_tuyển sinh.md',
       '/content/drive/MyDrive/Data/Admission/TDTU/Giới thiệu.md'],
      dtype=object)

In [ ]:
df2 = df[df['filename'] == docs[1]]
df2

id  \
35  2e76af70-6d29-4069-ba92-82286797d75e   
36  2f22d2d5-f663-4238-9576-ab8595c27a4d   
37  3317ac2e-8cc7-4395-8396-d00f4babc099   
38  332659e6-b1bf-45df-bfbe-2ad605a5d6e6   
39  34a360f3-90eb-4eb1-97e8-33bfb288c0c8   
40  36c549c1-0f16-4633-ba0b-6fc7a3c10f39   
41  370a71f5-058b-474a-a2a3-0c81aa6844d2   
42  3896a173-abc5-46f4-ab5c-551f261ef38e   
43  3ec93c12-47d6-4de5-8c65-7179d58f5bf6   
44  425a43ed-a1e7-406a-a456-5bb6da78c646   
45  4268c437-9e37-488d-8513-85dc9cb345d9   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [ ]:
import uuid
import time  # Import thư viện time để thêm khoảng nghỉ

data = []
for i in range(len(df2)):
    row = df2.iloc[i]
    text = row['text']
    question = generate_question(text)

    # Tạo UUID cho mỗi id_question
    id_question = str(uuid.uuid4())

    data.append({'id_question': id_question, 'question': question, 'id_corpus': row['id']})

new_df2 = pd.DataFrame(data)


In [ ]:
new_df2

,id_question,question,id_corpus
0,4b3bb8ab-6ed2-41c4-8ada-e1a1806453db,Bảng kết hợp này bao gồm dữ liệu của những năm nào?,2e76af70-6d29-4069-ba92-82286797d75e
1,160d5608-3e06-4f3c-af1f-f9558cab536c,Điểm chuẩn các ngành năm nào được liệt kê trong bảng trên?,2f22d2d5-f663-4238-9576-ab8595c27a4d
2,6f42de66-c1c2-4827-9ec8-da6f62d26fd1,Hạn chót nộp chứng chỉ IELTS cho thí sinh Việt Nam là ngày nào trong năm 2021?,3317ac2e-8cc7-4395-8396-d00f4babc099
3,e91edb9c-3264-4704-ad24-0916955cf891,Thí sinh trúng tuyển cần làm gì trên website từ 18/09/2021 đến 26/09/2021?,332659e6-b1bf-45df-bfbe-2ad605a5d6e6
4,6516f353-27fd-46a2-b004-b5102247423f,Hạn chót nộp Giấy chứng nhận kết quả thi THPT 2021 và phí nhập học tại TDTU là ngày nào?,34a360f3-90eb-4eb1-97e8-33bfb288c0c8
5,e187c290-bca9-4262-9d0b-c41b87020402,Thí sinh cần hoàn thành nộp phí nhập học trước 17h ngày 26/9/2021 bằng những phương thức nào?,36c549c1-0f16-4633-ba0b-6fc7a3c10f39
6,83f2a3ea-7641-4d9e-8a54-0ce2969440bc,Thí sinh nên ưu tiên phương thức nộp phí nào để tránh ảnh hưởng của dịch COVID-19?,370a71f5-058b-474a-a2a3-0c81aa6844d2
7,5ac321e5-75e9-4e01-a1fe-0088d48f248f,Thí sinh trúng tuyển năm 2021 tra cứu kết quả nhập học từ ngày nào?,3896a173-abc5-46f4-ab5c-551f261ef38e
8,6da35d9d-8dca-4fb2-ae5a-6dbf823a13c8,Thí sinh cần cập nhật thông tin và nộp hồ sơ sinh viên trong khoảng thời gian nào của năm 2021?,3ec93c12-47d6-4de5-8c65-7179d58f5bf6
9,5465c44f-47c2-4a8c-8a46-b63b2ac85d0b,"Nếu thí sinh khai báo sai thông tin, trường TDTU sẽ xử lý như thế nào?",425a43ed-a1e7-406a-a456-5bb6da78c646


In [ ]:
output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output1.xlsx"

try:
    new_df2.to_excel(output_path, index=False)  # Use index=False to avoid writing row indices
    print(f"DataFrame successfully exported to {output_path}")
except Exception as e:
    print(f"An error occurred while exporting the DataFrame: {e}")

DataFrame successfully exported to /content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output1.xlsx


# All code

In [ ]:
# prompt: from path /content/drive/MyDrive/Admission, lấy ra các file xlsx và in ra tên nó đi

import os

path = "/content/drive/MyDrive/University_Excel_File"

for filename in os.listdir(path):
    if filename.endswith(".xlsx"):
      print(filename)

PNTU.xlsx
OU.xlsx
NTTU.xlsx
HCMUTE.xlsx
HCMUS.xlsx
HCMUE.xlsx
FTU2.xlsx
VLU.xlsx
UMP.xlsx
UIT.xlsx
UFM.xlsx
UEH.xlsx
FPT.xlsx
TDTU.xlsx
BKU.xlsx


In [ ]:
excel_file_path = '/content/drive/MyDrive/University_Excel_File/TDTU.xlsx'
df = pd.read_excel(excel_file_path)
print(len(df))
# Lấy danh sách các giá trị unique trong cột 'filename'
docs = df['metadata'].unique()

180


In [ ]:
import pandas as pd
import uuid
import time

# Đọc file Excel
# excel_file_path = /content/drive/MyDrive/University_Excel_File/UEH.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/BKU.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/FPT.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/FTU2.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/HCMUE.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/HCMUTE.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/NTTU.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/OU.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/PNTU.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/UFM.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/UIT.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/UMP.xlsx'
# excel_file_path = '/content/drive/MyDrive/University_Excel_File/VLU.xlsx'

excel_file_path = '/content/drive/MyDrive/University_Excel_File/TDTU.xlsx'
df = pd.read_excel(excel_file_path)
print(len(df))
# Lấy danh sách các giá trị unique trong cột 'filename'
docs = df['metadata'].unique()

# Hàm generate_question như bạn đã định nghĩa
def generate_question(text, university):
    prompt = f"""
    Bạn là một trợ lý AI thông minh. Nhiệm vụ của bạn là tạo ra một câu hỏi liên quan trực tiếp đến nội dung của đoạn văn sau. Hãy đảm bảo câu hỏi tập trung vào các ý chính hoặc thông tin quan trọng trong đoạn văn.
    Câu hỏi ngắn gọn thôi, xúc tích vào. Tầm 20-35 token được rồi.
    Trường đại học cần hỏi: {university}
    Chú ý:
    - câu hỏi thường phải hỏi năm trong câu cụ thể có 4 năm là 2021, 2022, 2023, 2024.
    - Câu hỏi phải có tên trường,
    - nội dung câu phải liên quan đến các khía cạnh dưới nếu không liên quan thì có thể bỏ qua nội dung đó và không tạo câu hỏi:
      - Điều kiện tuyển sinh
      - Ngành học
      - Điểm chuẩn
      - Học bổng
      - Cơ sở vật chất
      - Môi trường học tập
      - Câu lạc bộ
      - Hoạt động ngoại khóa
      - Chính sách hỗ trợ sinh viên quốc tế
      - Thực tập

    Nội dung:
    "{text}"

    Câu hỏi:
    """
    response = model.generate_content(prompt, stream=False)
    return response.text.strip()

# Xử lý từng file (mỗi giá trị unique trong cột 'filename')
university = "TDTU"
all_data = []
for doc in docs:
    print(f"Đang xử lý file: {doc}")

    # Lọc DataFrame cho file hiện tại
    df1 = df[df['metadata'] == doc]

    # Khởi tạo danh sách dữ liệu tạm
    data = []

    # Tạo câu hỏi cho từng dòng trong DataFrame đã lọc
    for i in range(len(df1)):
        row = df1.iloc[i]
        text = row['page_content']
        question = generate_question(text, university)

        # Tạo UUID cho id_question
        id_question = str(uuid.uuid4())

        # Thêm kết quả vào danh sách
        data.append({'id_question': id_question, 'question': question, 'id_corpus': row['id'], 'filename': doc})

        # Nghỉ 10 giây sau mỗi 8 lần xử lý
        if (i + 1) % 7 == 0:
            print(f"Nghỉ 10 giây sau {i + 1} yêu cầu...")
            time.sleep(10)

    # Gộp dữ liệu tạm vào danh sách tổng
    all_data.extend(data)

# Tạo DataFrame tổng hợp từ tất cả dữ liệu
final_df = pd.DataFrame(all_data)

# Lưu kết quả ra file Excel

# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_ueh.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_bku.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_fpt.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_hcmue.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_hcmus.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_hcmute.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_nttu.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_ou.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_pntu.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_ufm.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_uit.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_ump.xlsx"
# output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_vlu.xlsx"
output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/output_tdtu.xlsx"

final_df.to_excel(output_path, index=False)

print(f"Kết quả đã được lưu vào {output_path}")


180
Đang xử lý file: {'source': '/content/drive/MyDrive/Data/Admission/TDTU/2021_tuyển sinh.md'}
Nghỉ 10 giây sau 7 yêu cầu...
Nghỉ 10 giây sau 14 yêu cầu...
Nghỉ 10 giây sau 21 yêu cầu...
Nghỉ 10 giây sau 28 yêu cầu...
Nghỉ 10 giây sau 35 yêu cầu...
Đang xử lý file: {'source': '/content/drive/MyDrive/Data/Admission/TDTU/2024_tuyển sinh.md'}
Nghỉ 10 giây sau 7 yêu cầu...
Nghỉ 10 giây sau 14 yêu cầu...
Nghỉ 10 giây sau 21 yêu cầu...
Nghỉ 10 giây sau 28 yêu cầu...
Nghỉ 10 giây sau 35 yêu cầu...
Đang xử lý file: {'source': '/content/drive/MyDrive/Data/Admission/TDTU/2023_tuyển sinh.md'}
Nghỉ 10 giây sau 7 yêu cầu...
Nghỉ 10 giây sau 14 yêu cầu...
Nghỉ 10 giây sau 21 yêu cầu...
Nghỉ 10 giây sau 28 yêu cầu...
Nghỉ 10 giây sau 35 yêu cầu...
Đang xử lý file: {'source': '/content/drive/MyDrive/Data/Admission/TDTU/Giới thiệu.md'}
Nghỉ 10 giây sau 7 yêu cầu...
Nghỉ 10 giây sau 14 yêu cầu...
Đang xử lý file: {'source': '/content/drive/MyDrive/Data/Admission/TDTU/2022_tuyển sinh.md'}
N

# Create all xlsx

In [ ]:
import os
import pandas as pd
import uuid
import time

# Đường dẫn đến thư mục chứa file Excel
input_path = "/content/drive/MyDrive/University_Excel_File/"
output_path = "/content/drive/MyDrive/TDTU_push /Data_QĐQC_TDTU/DataGroundTruth/"

# Hàm generate_question như bạn đã định nghĩa
def generate_question(text, university):
    prompt = f"""
    Bạn là một trợ lý AI thông minh. Nhiệm vụ của bạn là tạo ra một câu hỏi liên quan trực tiếp đến nội dung của đoạn văn sau. Hãy đảm bảo câu hỏi tập trung vào các ý chính hoặc thông tin quan trọng trong đoạn văn.
    Câu hỏi ngắn gọn thôi, xúc tích vào. Tầm 20-35 token được rồi.
    Trường đại học cần hỏi: {university}
    Chú ý:
    - câu hỏi thường phải hỏi năm trong câu cụ thể có 4 năm là 2021, 2022, 2023, 2024.
    - Câu hỏi phải có tên trường,
    - nội dung câu phải liên quan đến các khía cạnh dưới nếu không liên quan thì có thể bỏ qua nội dung đó và không tạo câu hỏi:
    - Điều kiện tuyển sinh
    - Ngành học
    - Điểm chuẩn
    - Học bổng
    - Cơ sở vật chất
    - Môi trường học tập
    - Câu lạc bộ
    - Hoạt động ngoại khóa
    - Chính sách hỗ trợ sinh viên quốc tế
    - Thực tập
    - Giao lưu quốc tế

    Nội dung:
    "{text}"

    Câu hỏi:
    """
    response = model.generate_content(prompt, stream=False)
    return response.text.strip()

# Lấy danh sách các file Excel từ thư mục
files = [f for f in os.listdir(input_path) if f.endswith(".xlsx")]
print(f"Đã tìm thấy {len(files)} file Excel: {files}")



Đã tìm thấy 15 file Excel: ['TDTU.xlsx', 'PNTU.xlsx', 'OU.xlsx', 'NTTU.xlsx', 'HCMUTE.xlsx', 'HCMUS.xlsx', 'HCMUE.xlsx', 'FTU2.xlsx', 'VLU.xlsx', 'UMP.xlsx', 'UIT.xlsx', 'UFM.xlsx', 'UEH.xlsx', 'BKU.xlsx', 'FPT.xlsx']


In [ ]:

all_files_data = []

# Xử lý từng file Excel
for file in files:
    university_name = os.path.splitext(file)[0]  # Tên trường từ tên file
    print(f"Đang xử lý file: {file} (Trường: {university_name})")

    # Đọc file Excel
    file_path = os.path.join(input_path, file)
    df = pd.read_excel(file_path)

    # Lấy danh sách các giá trị unique trong cột 'metadata'
    docs = df['metadata'].unique()

    file_data = []

    # Xử lý từng tài liệu trong file
    for doc in docs:
        print(f"Đang xử lý tài liệu: {doc}")

        # Lọc DataFrame cho file hiện tại
        df1 = df[df['metadata'] == doc]

        # Xử lý từng dòng
        for i in range(len(df1)):
            row = df1.iloc[i]
            text = row['page_content']
            question = generate_question(text, university_name)

            # Tạo UUID cho id_question
            id_question = str(uuid.uuid4())

            # Thêm kết quả vào danh sách
            file_data.append({
                'id_question': id_question,
                'question': question,
                'id_corpus': row['id'],
                'filename': doc
            })

            # Nghỉ 10 giây sau mỗi 7 lần xử lý
            if (i + 1) % 7 == 0:
                print(f"Nghỉ 10 giây sau {i + 1} yêu cầu...")
                time.sleep(10)

    all_files_data.extend(file_data)

    # Lưu kết quả tạm thời cho từng file
    temp_output_path = os.path.join(output_path, f"output_{university_name}.xlsx")
    pd.DataFrame(file_data).to_excel(temp_output_path, index=False)
    print(f"Đã lưu kết quả tạm thời cho {university_name} tại {temp_output_path}")

# Lưu kết quả tổng hợp ra file Excel
final_output_path = os.path.join(output_path, "output_all_universities.xlsx")
final_df = pd.DataFrame(all_files_data)
final_df.to_excel(final_output_path, index=False)

print(f"Kết quả tổng hợp đã được lưu vào {final_output_path}")

Đang xử lý file: TDTU.xlsx (Trường: TDTU)
Đang xử lý tài liệu: {'source': '/content/drive/MyDrive/Data/Admission/TDTU/2021_tuyển sinh.md'}
Nghỉ 10 giây sau 7 yêu cầu...
Nghỉ 10 giây sau 14 yêu cầu...
Nghỉ 10 giây sau 21 yêu cầu...
Nghỉ 10 giây sau 28 yêu cầu...
Nghỉ 10 giây sau 35 yêu cầu...
Đang xử lý tài liệu: {'source': '/content/drive/MyDrive/Data/Admission/TDTU/2024_tuyển sinh.md'}
Nghỉ 10 giây sau 7 yêu cầu...
Nghỉ 10 giây sau 14 yêu cầu...
Nghỉ 10 giây sau 21 yêu cầu...
Nghỉ 10 giây sau 28 yêu cầu...
Nghỉ 10 giây sau 35 yêu cầu...
Đang xử lý tài liệu: {'source': '/content/drive/MyDrive/Data/Admission/TDTU/2023_tuyển sinh.md'}
Nghỉ 10 giây sau 7 yêu cầu...
Nghỉ 10 giây sau 14 yêu cầu...
Nghỉ 10 giây sau 21 yêu cầu...
Nghỉ 10 giây sau 28 yêu cầu...
Nghỉ 10 giây sau 35 yêu cầu...
Đang xử lý tài liệu: {'source': '/content/drive/MyDrive/Data/Admission/TDTU/Giới thiệu.md'}
Nghỉ 10 giây sau 7 yêu cầu...
Nghỉ 10 giây sau 14 yêu cầu...
Đang xử lý tài liệu: {'source': '/content/